In [21]:
import json
import random
import spacy
from spacy.training.example import Example
from sklearn.metrics import precision_recall_fscore_support
from pathlib import Path

# Caminho para o dataset anotado
DATA_PATH = Path("../data/annotations/iphone_auto_annotations.jsonl")
MODEL_OUTPUT = Path("../models/spacy_iphone_model")

# ----------------------------
# 1️ Carregar dataset JSONL
# ----------------------------
def load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            record = json.loads(line.strip())
            data.append(record)
    return data

data = load_jsonl(DATA_PATH)
print(f"Total de exemplos carregados: {len(data)}")

# ----------------------------
# 2️ Separar em treino e teste (80/20)
# ----------------------------
random.shuffle(data)
split = int(len(data) * 0.8)
train_data = data[:split]
test_data = data[split:]

print(f"Treino: {len(train_data)} exemplos | Teste: {len(test_data)} exemplos")

# ----------------------------
# 3️ Preparar formato spaCy
# ----------------------------
def to_spacy_format(dataset):
    """Converte para o formato [(text, {"entities": [...]})]"""
    formatted = []
    for item in dataset:
        text = item["text"]
        entities = item["entities"]
        formatted.append((text, {"entities": [tuple(e) for e in entities]}))
    return formatted

train_data_spacy = to_spacy_format(train_data)
test_data_spacy = to_spacy_format(test_data)

# ----------------------------
# 4️ Criar modelo spaCy NER
# ----------------------------
nlp = spacy.blank("pt")  
ner = nlp.add_pipe("ner")

# Adiciona os rótulos (labels) detectados no dataset
labels = set(ent[2] for item in data for ent in item["entities"])
for label in labels:
    ner.add_label(label)

# ----------------------------
# 5️ Treinar modelo
# ----------------------------
n_iter = 30
optimizer = nlp.begin_training()
losses = {}

print("🚀 Treinando modelo NER...")
for itn in range(n_iter):
    random.shuffle(train_data_spacy)
    losses = {}
    for text, annotations in train_data_spacy:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.2, sgd=optimizer, losses=losses)
    if (itn + 1) % 5 == 0:
        print(f"Iteração {itn+1}/{n_iter}, Loss: {losses.get('ner', 0):.4f}")

print("Treinamento concluído!")

# ----------------------------
# 6️ Avaliação no conjunto de teste
# ----------------------------
true_labels, pred_labels = [], []

for text, annotations in test_data_spacy:
    doc = nlp(text)
    true_ents = [ent[2] for ent in annotations["entities"]]
    pred_ents = [ent.label_ for ent in doc.ents]
    # Adiciona para métricas (apenas tipos de entidade)
    for lbl in labels:
        true_labels.append(lbl in true_ents)
        pred_labels.append(lbl in pred_ents)

precision, recall, f1, _ = precision_recall_fscore_support(
    true_labels, pred_labels, average="binary"
)

print("\nAvaliação:")
print(f"Precisão: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")

# ----------------------------
# 7️ Salvar modelo treinado
# ----------------------------
MODEL_OUTPUT.mkdir(parents=True, exist_ok=True)
nlp.to_disk(MODEL_OUTPUT)
print(f"Modelo salvo em: {MODEL_OUTPUT}")

# ----------------------------
# 8️ Teste de predição
# ----------------------------
sample_text = "iPhone 14 Plus 128GB Azul"
doc = nlp(sample_text)
print("\n🔍 Exemplo de predição:")
for ent in doc.ents:
    print(f"{ent.text} → {ent.label_}")


Total de exemplos carregados: 336
Treino: 268 exemplos | Teste: 68 exemplos
🚀 Treinando modelo NER...


/usr/local/python/3.12.1/lib/python3.12/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "iPhone XR Apple 64GB, Tela Retina LCD de 6,1”, iOS..." with entities "[(0, 6, 'CATEGORIA'), (10, 15, 'MARCA'), (16, 20, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Apple iPhone 14 Pro 256GB Preto-espacial" with entities "[(0, 5, 'MARCA'), (6, 12, 'CATEGORIA'), (13, 19, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030

Iteração 5/30, Loss: 90.3136
Iteração 10/30, Loss: 63.0253
Iteração 15/30, Loss: 41.2146
Iteração 20/30, Loss: 38.6879
Iteração 25/30, Loss: 46.7217
Iteração 30/30, Loss: 36.3658
Treinamento concluído!

Avaliação:
Precisão: 0.993
Recall: 0.967
F1-score: 0.980
Modelo salvo em: ../models/spacy_iphone_model

🔍 Exemplo de predição:
iPhone → CATEGORIA
14 → MODELO
128GB → MEMORIA
Azul → COR


In [ ]:
# ----------------------------
# 8️ Teste de predição (usando o modelo salvo)
# ----------------------------
import spacy

# Carrega o modelo salvo
MODEL_PATH = "../models/spacy_iphone_model"
nlp_loaded = spacy.load(MODEL_PATH)

# Frase de teste
sample_text = "iphone 18 Victor 128gb oiiiii cor verde"
doc = nlp_loaded(sample_text)

print("\nExemplo de predição (modelo carregado):")
for ent in doc.ents:
    print(f"{ent.text} → {ent.label_}")



🔍 Exemplo de predição (modelo carregado):
iphone → CATEGORIA
18 → MODELO
128gb → MEMORIA
verde → COR


c:\Users\victor.miguel\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'pt_pipeline' (0.0.0) was trained with spaCy v3.8.7 and may not be 100% compatible with the current version (3.8.6). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
